# Project Overview

Perform data analysis and data preparation tasks to train a Linear Regression model to predict future ROI (Return On Investment) of variable ad spend budgets across multiple channels including search, video, social media, and email using Snowpark for Python, Snowpark ML and Streamlit. By the end of the session, you will have an interactive web application deployed visualizing the ROI of different allocated advertising spend budgets.

***Prerequisite**: Before proceeding with this Notebook, you must first successfully run through Snowpark_For_Python_DE.ipynb.*
## Machine Learning

In this Notebook, we will focus on Machine Learning in Snowflake using Snowpark for Python.

- Establish secure connection to Snowflake
- Load features and target from Snowflake table into Snowpark DataFrame
- Prepare features for model training
- Train ML model using Snowpark ML in Snowflake and upload the model to Snowflake stage
- Register ML model and use it for inference from Snowpark ML Model Registry (currently in public preview)
    - Note: The ML Model is called from the Streamlit Apps. See [Snowpark_Streamlit_Revenue_Prediction.py](https://github.com/Snowflake-Labs/sfguide-getting-started-dataengineering-ml-snowpark-python/blob/5dd84401337fab20fd8bf95eee3c70b0b2d11809//Snowpark_Streamlit_Revenue_Prediction.py) and [Snowpark_Streamlit_Revenue_Prediction_SiS.py](https://github.com/Snowflake-Labs/sfguide-getting-started-dataengineering-ml-snowpark-python/blob/5dd84401337fab20fd8bf95eee3c70b0b2d11809//Snowpark_Streamlit_Revenue_Prediction_SiS.py)
  
For environment setup including loading data into Snowflake tables, and step-by-step instructions, please refer to the [QuickStart Guide](https://quickstarts.snowflake.com/guide/getting_started_with_dataengineering_ml_using_snowpark_python/index.html#0).


### Import Libraries

In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION

# Snowpark ML
from snowflake.ml.modeling.compose import ColumnTransformer
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import PolynomialFeatures, StandardScaler
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
from snowflake.ml.version import VERSION as ml_version

# Misc
#import pandas as pd
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

#### Establish Secure Connection to Snowflake

Using the Snowpark Python API, it’s quick and easy to establish a secure connection between Snowflake and Notebook.

Note: Other connection options include Username/Password, MFA, OAuth, Okta, SSO

TIP: Learn more about [Session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/session) object.


In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
print('Snowpark ML version         : {}'.format(ml_version))

User                        : TRAININGSNOWFLAKE
Role                        : "ACCOUNTADMIN"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L1"
Snowflake version           : 8.12.2
Snowpark for Python version : 1.13.0
Snowpark ML version         : 1.2.3


### Features and Target

At this point we are ready to perform the following actions to save features and target for model training.

- Delete rows with missing values
- Exclude columns we don't need for modeling
- Save features into a Snowflake table called MARKETING_BUDGETS_FEATURES



In [3]:
# Load data
snow_df_spend_and_revenue_per_month = session.table('spend_and_revenue_per_month')

# Delete rows with missing values
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.dropna()

# Exclude columns we don't need for modeling
snow_df_spend_and_revenue_per_month = snow_df_spend_and_revenue_per_month.drop(['YEAR','MONTH'])

# Save features into a Snowflake table call MARKETING_BUDGETS_FEATURES
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('MARKETING_BUDGETS_FEATURES')
snow_df_spend_and_revenue_per_month.show()

---------------------------------------------------------------------
|"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |"REVENUE"   |
---------------------------------------------------------------------
|516431           |517618          |516729   |517208   |3264300.11  |
|506497           |504679          |501098   |501947   |3208482.33  |
|522780           |521395          |522762   |518405   |3311966.98  |
|519959           |520537          |520685   |521584   |3311752.81  |
|507211           |507404          |511364   |507363   |3208563.06  |
|518942           |520863          |522768   |519950   |3334028.46  |
|505715           |505221          |505292   |503748   |3185894.64  |
|520148           |520711          |521427   |520724   |3334570.96  |
|522151           |518635          |520583   |521167   |3316455.44  |
|467736           |474679          |469856   |469784   |2995042.21  |
---------------------------------------------------------------------



### Model Training using Snowpark ML in Snowflake

Learn more about Snowpark ML.

NOTE: For workloads that require a large amount of memory and compute resources, consider using [Snowpark-Optimized Warehouses](https://docs.snowflake.com/en/developer-guide/snowpark/python/python-snowpark-training-ml#snowpark-optimized-warehouses).


In [4]:
CROSS_VALIDATION_FOLDS = 10
POLYNOMIAL_FEATURES_DEGREE = 2

# Create train and test Snowpark DataDrames
train_df, test_df = session.table("MARKETING_BUDGETS_FEATURES").random_split(weights=[0.8, 0.2], seed=0)

# Preprocess the Numeric columns
# We apply PolynomialFeatures and StandardScaler preprocessing steps to the numeric columns
# NOTE: High degrees can cause overfitting.
numeric_features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
numeric_transformer = Pipeline(steps=[('poly',PolynomialFeatures(degree = POLYNOMIAL_FEATURES_DEGREE)),('scaler', StandardScaler())])

# Combine the preprocessed step together using the Column Transformer module
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

# The next step is the integrate the features we just preprocessed with our Machine Learning algorithm to enable us to build a model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),('classifier', LinearRegression())])
parameteres = {}

# Use GridSearch to find the best fitting model based on number_of_folds folds
model = GridSearchCV(
    estimator=pipeline,
    param_grid=parameteres,
    cv=CROSS_VALIDATION_FOLDS,
    label_cols=["REVENUE"],
    output_cols=["PREDICTED_REVENUE"],
    verbose=2
)

# Fit and Score
model.fit(train_df)
train_r2_score = model.score(train_df)
test_r2_score = model.score(test_df)

# R2 score on train and test datasets
print(f"R2 score on Train : {train_r2_score}")
print(f"R2 score on Test  : {test_r2_score}")

R2 score on Train : 0.995107670224394
R2 score on Test  : 0.9069317995589423


### Log Trained Model to Snowflake Model Registry

The Model Registry allows to store models as objects in a schema in Snowflake. Note that by default the database and schema of the session is used.

Learn more about [Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops-model-registry).


In [5]:
registry = Registry(session)
MODEL_NAME = "PREDICT_ROI"

In [6]:
# NOTE: If you try to log the model with the same name, you may get "ValueError: (0000) Model PREDICT_ROI version v1 already existed." error. 
# If that's the case, uncomment and run this cell.

# registry.delete_model(MODEL_NAME)

In [7]:
mv = registry.log_model(model,
                        model_name=MODEL_NAME,
                        version_name="v1",
                        metrics={"R2_train": train_r2_score, "R2_test":test_r2_score},
                        comment='Model pipeline to predict revenue',
                        options={"embed_local_ml_library": True}
                    )

### View Logged Model in Snowflake Model Registry

In [8]:
registry.show_models()

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-04-02 09:13:03.543000-07:00,PREDICT_ROI,DASH_DB,DASH_SCHEMA,None,ACCOUNTADMIN,V1,"[""V1""]"


### Inference

Once the model is logged we can use it for inference on new data.

First we will create a Snowpark DataFrame with some sample data and then call the logged model to get new predictions. Note: we will handle negative values in our Streamlit application.

In [9]:
test_df = session.create_dataframe([[250000,250000,200000,450000],[500000,500000,500000,500000],[8500,9500,2000,500]], 
                                    schema=['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL'])
mv.run(test_df, function_name='predict').show()

------------------------------------------------------------------------------
|"SEARCH_ENGINE"  |"SOCIAL_MEDIA"  |"VIDEO"  |"EMAIL"  |"PREDICTED_REVENUE"  |
------------------------------------------------------------------------------
|8500             |9500            |2000     |500      |-107982.7042556526   |
|250000           |250000          |200000   |450000   |-12735711.132665249  |
|500000           |500000          |500000   |500000   |3180176.6792253554   |
------------------------------------------------------------------------------



In [10]:
root_task = tasks["DE_PIPELINE_DAG"]
root_task.suspend()

NameError: name 'tasks' is not defined